# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#preprocessing and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler


# ML
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

#DL
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2

#Statistical test
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import ttest_rel


# Hide Warnings
import warnings

warnings.filterwarnings("ignore")

from IPython.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.output_result { max-width:95% !important; }</style>"))

In [ ]:
df_complete = pd.read_feather("complete_data.feather")
df_complete["date"] = pd.to_datetime(df_complete["date"], format="%Y-%m-%d")
df_complete.reset_index(drop=True, inplace=True)
df_complete.set_index("date", inplace=True, drop=True)
df_complete

,arpa_CO,s5p_CO,cams_CO,r,r_pre,tp,ssr,str,t2m,sp,...,tp_pre,ssr_pre,str_pre,t2m_pre,sp_pre,blh_pre,wind_speed_pre,wind_dir_pre,lai_hv_pre,lai_lv_pre
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,0.618750,0.035791,4.095564e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01,1.812500,0.034532,3.810000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01,1.400000,0.034532,3.717811e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01,0.825000,0.035447,3.616589e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01,1.243750,0.036033,3.614726e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,1.373684,0.038286,NaN,NaN,NaN,1.808715e-07,257601.098259,-224314.557407,276.479340,101519.943595,...,0.0,13306.114614,-215544.141199,276.518430,101354.229813,21.353957,1.115551,283.396891,1.922135,1.638666
2024-12-31,1.647368,0.038143,NaN,NaN,NaN,1.049269e-07,260016.302506,-227914.900344,276.385801,101132.118861,...,0.0,13388.970664,-219750.408036,276.731064,100971.043009,20.802328,1.136056,280.839577,1.963621,1.607436
2024-12-31,2.026316,0.037352,NaN,NaN,NaN,9.712973e-08,261847.113250,-231467.654353,276.434759,100798.676379,...,0.0,13364.250011,-222639.835045,276.858498,100641.185382,20.326394,1.105231,267.814780,1.960006,1.625828


In [3]:
df_complete.dtypes

arpa_CO           float64
s5p_CO            float64
cams_CO           float64
r                 float64
r_pre             float64
tp                float64
ssr               float64
str               float64
t2m               float64
sp                float64
blh               float64
wind_speed        float64
wind_dir          float64
lai_hv            float64
lai_lv            float64
tp_pre            float64
ssr_pre           float64
str_pre           float64
t2m_pre           float64
sp_pre            float64
blh_pre           float64
wind_speed_pre    float64
wind_dir_pre      float64
lai_hv_pre        float64
lai_lv_pre        float64
dtype: object

# Feature engineering

## Wind Direction Average

In [5]:
# Group by date and compute vector-averaged wind directions
def vector_average_angle(angles_deg):
    angles_rad = np.deg2rad(angles_deg)
    sin_avg = np.mean(np.sin(angles_rad))
    cos_avg = np.mean(np.cos(angles_rad))
    angle_rad_avg = np.arctan2(sin_avg, cos_avg)
    angle_deg_avg = (np.rad2deg(angle_rad_avg) + 360) % 360
    return angle_deg_avg

df_complete_wind_dir = df_complete.groupby("date").agg(
    wind_dir=pd.NamedAgg(column="wind_dir", aggfunc=vector_average_angle),
    wind_dir_pre=pd.NamedAgg(column="wind_dir_pre", aggfunc=vector_average_angle)
).reset_index()

In [6]:
df_complete_wind_dir

,date,wind_dir,wind_dir_pre
0,2019-01-01,NaN,NaN
1,2019-01-02,NaN,NaN
2,2019-01-03,289.699303,137.417828
3,2019-01-04,204.734891,259.681814
4,2019-01-05,258.634541,263.237834
...,...,...,...
2187,2024-12-27,261.868878,246.178078
2188,2024-12-28,246.633360,264.293779
2189,2024-12-29,233.218724,239.620729
2190,2024-12-30,230.717246,269.375521


## Wind Cyclical Encoding

In [7]:
def add_cyclical_features(df, column, max_value=360):
    df[f"{column}_sin"] = np.sin(2 * np.pi * df[column] / max_value)
    df[f"{column}_cos"] = np.cos(2 * np.pi * df[column] / max_value)
    return df

df_complete_wind_dir = add_cyclical_features(df_complete_wind_dir, "wind_dir")
df_complete_wind_dir = add_cyclical_features(df_complete_wind_dir, "wind_dir_pre")
#drop the original angle columns
df_complete_wind_dir = df_complete_wind_dir.drop(columns=["wind_dir", "wind_dir_pre"])
df_complete_wind_dir

,date,wind_dir_sin,wind_dir_cos,wind_dir_pre_sin,wind_dir_pre_cos
0,2019-01-01,NaN,NaN,NaN,NaN
1,2019-01-02,NaN,NaN,NaN,NaN
2,2019-01-03,-0.941475,0.337084,0.676647,-0.736308
3,2019-01-04,-0.418420,-0.908254,-0.983828,-0.179114
4,2019-01-05,-0.980390,-0.197066,-0.993043,-0.117748
...,...,...,...,...,...
2187,2024-12-27,-0.989947,-0.141439,-0.914805,-0.403895
2188,2024-12-28,-0.917986,-0.396613,-0.995045,-0.099428
2189,2024-12-29,-0.800927,-0.598762,-0.862697,-0.505722
2190,2024-12-30,-0.774031,-0.633148,-0.999941,-0.010899


## Average by date

In [8]:
df_complete

,arpa_CO,s5p_CO,cams_CO,r,r_pre,tp,ssr,str,t2m,sp,...,tp_pre,ssr_pre,str_pre,t2m_pre,sp_pre,blh_pre,wind_speed_pre,wind_dir_pre,lai_hv_pre,lai_lv_pre
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,0.618750,0.035791,4.095564e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01,1.812500,0.034532,3.810000e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01,1.400000,0.034532,3.717811e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01,0.825000,0.035447,3.616589e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01,1.243750,0.036033,3.614726e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31,1.373684,0.038286,NaN,NaN,NaN,1.808715e-07,257601.098259,-224314.557407,276.479340,101519.943595,...,0.0,13306.114614,-215544.141199,276.518430,101354.229813,21.353957,1.115551,283.396891,1.922135,1.638666
2024-12-31,1.647368,0.038143,NaN,NaN,NaN,1.049269e-07,260016.302506,-227914.900344,276.385801,101132.118861,...,0.0,13388.970664,-219750.408036,276.731064,100971.043009,20.802328,1.136056,280.839577,1.963621,1.607436
2024-12-31,2.026316,0.037352,NaN,NaN,NaN,9.712973e-08,261847.113250,-231467.654353,276.434759,100798.676379,...,0.0,13364.250011,-222639.835045,276.858498,100641.185382,20.326394,1.105231,267.814780,1.960006,1.625828


In [9]:
df_complete=df_complete.drop(
    columns=["wind_dir", "wind_dir_pre"])
df_complete = df_complete.groupby("date").mean()
df_complete

,arpa_CO,s5p_CO,cams_CO,r,r_pre,tp,ssr,str,t2m,sp,...,lai_lv,tp_pre,ssr_pre,str_pre,t2m_pre,sp_pre,blh_pre,wind_speed_pre,lai_hv_pre,lai_lv_pre
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1.180000,0.035267,3.770938e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,0.891579,0.032974,2.824356e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,0.606316,0.036171,1.889109e-07,46.641597,37.779809,4.738731e-09,298597.147719,-371299.779479,277.819891,100262.874109,...,1.618758,2.369910e-07,16398.122648,-378144.578206,280.460087,100143.018120,1122.116296,5.528460,1.963685,1.619105
2019-01-04,1.064211,NaN,3.196309e-07,51.410032,43.903486,4.231597e-07,295073.856831,-307369.054841,273.828648,100522.088216,...,1.618092,0.000000e+00,16676.530737,-306054.917471,274.996017,100344.389126,134.655960,1.620950,1.963370,1.618413
2019-01-05,1.576842,0.037833,3.064936e-07,59.479203,53.215448,1.964209e-07,272394.035475,-261829.277860,274.369225,99999.784231,...,1.617425,0.000000e+00,16718.959529,-218960.789968,274.672809,100480.141480,19.249516,0.790367,1.963062,1.617762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,1.031579,0.034768,NaN,NaN,NaN,9.524280e-10,284643.225136,-278471.518188,275.053108,101081.016524,...,1.623439,0.000000e+00,11681.295304,-268613.976806,275.409244,101043.542665,26.209431,1.631343,1.965906,1.623777
2024-12-28,1.041053,0.033513,NaN,NaN,NaN,9.524280e-10,282490.875944,-256997.660214,276.680927,100637.889865,...,1.622770,0.000000e+00,11980.756092,-252283.423520,275.556256,100843.841802,29.807412,1.368907,1.965581,1.623096
2024-12-29,1.366316,0.035366,NaN,NaN,NaN,0.000000e+00,286180.455282,-265531.984307,277.360834,100465.890162,...,1.622101,0.000000e+00,12186.128974,-228643.041695,277.948905,100428.646260,19.577048,1.126425,1.965265,1.622443


## Merge wind

In [10]:
df_complete = df_complete.merge(df_complete_wind_dir, on="date")
df_complete.set_index("date", drop=True, inplace=True)
df_complete

,arpa_CO,s5p_CO,cams_CO,r,r_pre,tp,ssr,str,t2m,sp,...,t2m_pre,sp_pre,blh_pre,wind_speed_pre,lai_hv_pre,lai_lv_pre,wind_dir_sin,wind_dir_cos,wind_dir_pre_sin,wind_dir_pre_cos
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,1.180000,0.035267,3.770938e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,0.891579,0.032974,2.824356e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,0.606316,0.036171,1.889109e-07,46.641597,37.779809,4.738731e-09,298597.147719,-371299.779479,277.819891,100262.874109,...,280.460087,100143.018120,1122.116296,5.528460,1.963685,1.619105,-0.941475,0.337084,0.676647,-0.736308
2019-01-04,1.064211,NaN,3.196309e-07,51.410032,43.903486,4.231597e-07,295073.856831,-307369.054841,273.828648,100522.088216,...,274.996017,100344.389126,134.655960,1.620950,1.963370,1.618413,-0.418420,-0.908254,-0.983828,-0.179114
2019-01-05,1.576842,0.037833,3.064936e-07,59.479203,53.215448,1.964209e-07,272394.035475,-261829.277860,274.369225,99999.784231,...,274.672809,100480.141480,19.249516,0.790367,1.963062,1.617762,-0.980390,-0.197066,-0.993043,-0.117748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,1.031579,0.034768,NaN,NaN,NaN,9.524280e-10,284643.225136,-278471.518188,275.053108,101081.016524,...,275.409244,101043.542665,26.209431,1.631343,1.965906,1.623777,-0.989947,-0.141439,-0.914805,-0.403895
2024-12-28,1.041053,0.033513,NaN,NaN,NaN,9.524280e-10,282490.875944,-256997.660214,276.680927,100637.889865,...,275.556256,100843.841802,29.807412,1.368907,1.965581,1.623096,-0.917986,-0.396613,-0.995045,-0.099428
2024-12-29,1.366316,0.035366,NaN,NaN,NaN,0.000000e+00,286180.455282,-265531.984307,277.360834,100465.890162,...,277.948905,100428.646260,19.577048,1.126425,1.965265,1.622443,-0.800927,-0.598762,-0.862697,-0.505722


In [11]:
# Solar-Thermal Contrast
df_complete["solar_thermal_contrast"] = (df_complete["ssr"] - df_complete["str"]) / (df_complete["ssr"] + df_complete["str"])

In [12]:
#normalized_CO = s5p_CO / blh
df_complete["CO_per_blh"] = df_complete["s5p_CO"] / df_complete["blh"]

In [ ]:
# Drop rows with NaN 
df_complete = df_complete.dropna()

In [14]:
df_complete

,arpa_CO,s5p_CO,cams_CO,r,r_pre,tp,ssr,str,t2m,sp,...,blh_pre,wind_speed_pre,lai_hv_pre,lai_lv_pre,wind_dir_sin,wind_dir_cos,wind_dir_pre_sin,wind_dir_pre_cos,solar_thermal_contrast,CO_per_blh
date,,,,,,,,,,,,,,,,,,,,,
2019-01-03,0.606316,0.036171,1.889109e-07,46.641597,37.779809,4.738731e-09,298597.147719,-371299.779479,277.819891,100262.874109,...,1122.116296,5.528460,1.963685,1.619105,-0.941475,0.337084,0.676647,-0.736308,-9.214205,0.000070
2019-01-05,1.576842,0.037833,3.064936e-07,59.479203,53.215448,1.964209e-07,272394.035475,-261829.277860,274.369225,99999.784231,...,19.249516,0.790367,1.963062,1.617762,-0.980390,-0.197066,-0.993043,-0.117748,50.566547,0.000385
2019-01-06,1.402105,0.035538,2.860080e-07,58.450944,60.573325,1.658556e-08,304366.605549,-302604.884703,279.191344,99666.374389,...,105.204894,2.905333,1.962768,1.617083,-0.764365,0.644784,-0.897872,0.440257,344.533296,0.000147
2019-01-07,1.468421,0.039746,2.828122e-07,55.165493,52.032681,0.000000e+00,300842.199608,-309747.287126,278.250179,100141.215810,...,264.564240,3.930619,1.962438,1.616421,-0.484231,-0.874940,-0.507895,0.861419,-68.566366,0.000285
2019-01-09,1.129474,0.033214,2.957438e-07,69.126679,72.761415,6.130445e-07,281774.024489,-306795.023985,276.935752,98574.571534,...,48.843135,1.950357,1.961802,1.615092,-0.910653,0.413172,-0.861429,0.507878,-23.523003,0.000076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-26,0.820000,0.034916,2.476492e-07,85.774530,85.915377,2.501625e-04,140350.953374,-59011.873668,281.742355,99978.122368,...,54.885669,0.682863,1.995360,1.687291,-0.416424,-0.909170,-0.673899,-0.738824,2.451009,0.000291
2024-11-27,0.837895,0.032648,2.918627e-07,87.993952,84.722713,1.431004e-05,171169.725732,-92458.869431,282.147933,100065.732253,...,43.640718,0.852439,1.994008,1.684388,-0.860747,-0.509033,0.963696,-0.267003,3.349330,0.000289
2024-11-28,0.828421,0.033417,2.759996e-07,92.100006,83.978641,2.415389e-06,189747.919439,-99616.343867,281.623480,100063.706832,...,25.469618,0.945131,1.992677,1.681518,-0.978663,-0.205471,-0.939450,-0.342686,3.210465,0.000275


# GEDA

In [ ]:
# Graph ground CO complete timeseries
fig = go.Figure()
fig.add_trace(
    go.Scatter(y=df_complete["arpa_CO"], x=df_complete.index, name="arpa_CO")
)
fig.update_xaxes(
    title_text="Date",
    showgrid=True,
    gridwidth=1,
    gridcolor="LightGray",
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="gray",
)
fig.update_yaxes(
    title_text="CO<sub></sub> [mg/m<sup>3</sup>]",
    showgrid=True,
    gridwidth=1,
    gridcolor="LightGray",
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="gray",
)
fig.update_layout(paper_bgcolor="rgba(0,0,0,0)", plot_bgcolor="rgba(0,0,0,0)")
fig.show()

# Remove unimportant features

In [16]:
columns_to_keep=[
'arpa_CO',
'cams_CO',
 'blh_pre',
 'lai_lv_pre',
 't2m',
 'sp_pre',
 'lai_lv',
 'ssr',
 'r_pre',
 'tp_pre',
 'CO_per_blh',
 't2m_pre',
 'str',
 'blh',
 's5p_CO',
 'r',
 'sp']
# Filter all datasets to use only the selected columns
df_complete= df_complete[columns_to_keep]



# Parameters

In [ ]:
svr_params = {
    'kernel':'rbf',
    'C': 2,
    'gamma': 0.01,
    'epsilon': 0.2
}

mlp_params= {
    'hidden_layer_sizes': (128, 64),
    'activation': 'tanh',          
    'solver': 'adam',
    'alpha': 0.0005,
    'learning_rate_init': 0.0005,
    'learning_rate': 'adaptive',
    'batch_size': 128,
    'max_iter': 5000,
    'tol': 1e-6,
    'early_stopping': True,
    'validation_fraction': 0.2,
    'n_iter_no_change': 25
}
rf_params = {
    'n_estimators': 500,
    'max_depth': 5,
    'min_samples_split':25,
    'min_samples_leaf': 25,
    'max_features':0.6,
    'bootstrap': True,
    'max_samples': 0.6
}
xgb_params = {
    'n_estimators': 500,
    'max_depth': 5,                 
    'learning_rate': 0.02,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'gamma':1.5,
    'min_child_weight': 10,
    'reg_alpha': 12,
    'reg_lambda': 12,
}
dt_params = {
    'max_depth': 4,                  
    'min_samples_split': 20,         
    'min_samples_leaf': 20,          
    'max_features': 0.6,             
    'criterion': 'absolute_error',
    'ccp_alpha': 0.008,              
}
gbr_params = {
    'n_estimators': 350,
    'learning_rate': 0.03,
    'max_depth': 2,
    'min_samples_split': 50,
    'min_samples_leaf': 50,
    'max_features': 0.5,
    'subsample': 0.5
}

# Model Training and Testing

In [ ]:
# Initialize storage for results
results = {
    'SVR': [],
    'MLP': [],
    'SVR_MLP': [],
    'DenseAttNet': [],
    'RF': [],
    'DT': [],
    'GBR': [],
    'XGB': []
}
# Define a custom attention layer
def attention_layer(inputs):
    # Compute attention scores
    attention_probs = Dense(128, activation='softmax', name="AttentionScores")(inputs)
    # Apply attention to the inputs
    attention_output = Multiply(name="AttentionApplied")([inputs, attention_probs])
    return attention_output


# Define neural network architecture function
def create_enhanced_model(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(128, activation='relu',kernel_regularizer=l2(0.001))(input_layer)  # L2 regularizatio
    x = Dropout(0.1)(x)
    # Add attention after the first hidden layer
    x = attention_layer(x)
    x = Dense(64, activation='relu',kernel_regularizer=l2(0.001))(x)
    x = Dropout(0.1)(x)
    output_layer = Dense(1)(x)  # Regression output
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(
    optimizer=Adam(learning_rate=0.0005),loss='mse',metrics=['mae'])
    return model

for i in range(1,21):  # 20 repetitions
    print(f"\n=== Iteration {i+1}/20 ===")
    
    # 1. Create new random split
    train, test = train_test_split(
        df_complete, test_size=0.2, random_state=i, shuffle=True
    )
    
    # 2. Preprocessing (using current training set only)
    # Identify skewed features
    skewness = train.drop(columns=['arpa_CO']).skew()
    high_skew_cols = skewness[abs(skewness) > 0.5].index.tolist()
    
    # Apply Yeo-Johnson
    if high_skew_cols:
        yeo = PowerTransformer(method='yeo-johnson')
        yeo.fit(train[high_skew_cols])
        train[high_skew_cols] = yeo.transform(train[high_skew_cols])
        test[high_skew_cols] = yeo.transform(test[high_skew_cols])
    
    # 3. Prepare data
    def prepare_sets(df):
        y = df[['arpa_CO']].values
        X = df.drop(columns=['arpa_CO']).values
        return X, y

    train_X, train_y = prepare_sets(train)
    test_X, test_y = prepare_sets(test)
    
    # Calculate std of test_y for NRMSE (before scaling!)
    test_y_std = np.std(test_y)

    # 4. Scale data (fit on training only)
    X_scaler = StandardScaler()
    y_scaler = StandardScaler()
    
    X_scaler.fit(train_X)
    y_scaler.fit(train_y)
    
    s_train_X = X_scaler.transform(train_X)
    s_train_y = y_scaler.transform(train_y).ravel()
    s_test_X = X_scaler.transform(test_X)
    s_test_y = y_scaler.transform(test_y).ravel()
    
    # 5. Train and evaluate models
    # Random Forest (RF)
    rf_model = RandomForestRegressor(random_state=i, **rf_params)  # Add your params
    rf_model.fit(s_train_X, s_train_y)
    yhat_rf = rf_model.predict(s_test_X)
    inv_yhat_rf = y_scaler.inverse_transform(yhat_rf.reshape(-1, 1))
    rmse_rf = np.sqrt(mean_squared_error(test_y, inv_yhat_rf))
    nrmse_rf = rmse_rf / test_y_std
    results['RF'].append(nrmse_rf)

    # Decision Tree (DT)
    dt_model = DecisionTreeRegressor(random_state=i, **dt_params)  # Add your params
    dt_model.fit(s_train_X, s_train_y)
    yhat_dt = dt_model.predict(s_test_X)
    inv_yhat_dt = y_scaler.inverse_transform(yhat_dt.reshape(-1, 1))
    rmse_dt = np.sqrt(mean_squared_error(test_y, inv_yhat_dt))
    nrmse_dt = rmse_dt / test_y_std
    results['DT'].append(nrmse_dt)

    # Gradient Boosting (GBR)
    gbr_model = GradientBoostingRegressor(random_state=i, **gbr_params)  # Add your params
    gbr_model.fit(s_train_X, s_train_y)
    yhat_gbr = gbr_model.predict(s_test_X)
    inv_yhat_gbr = y_scaler.inverse_transform(yhat_gbr.reshape(-1, 1))
    rmse_gbr = np.sqrt(mean_squared_error(test_y, inv_yhat_gbr))
    nrmse_gbr = rmse_gbr / test_y_std
    results['GBR'].append(nrmse_gbr)

    # XGBoost (XGB)
    xgb_model = XGBRegressor(random_state=i, **xgb_params)  # Add your params
    xgb_model.fit(s_train_X, s_train_y)
    yhat_xgb = xgb_model.predict(s_test_X)
    inv_yhat_xgb = y_scaler.inverse_transform(yhat_xgb.reshape(-1, 1))
    rmse_xgb = np.sqrt(mean_squared_error(test_y, inv_yhat_xgb))
    nrmse_xgb = rmse_xgb / test_y_std
    results['XGB'].append(nrmse_xgb)
    # SVR
    svr_model = SVR(**svr_params)  # Add your pre-tuned parameters
    svr_model.fit(s_train_X, s_train_y)
    yhat_svr = svr_model.predict(s_test_X)
    inv_yhat_svr = y_scaler.inverse_transform(yhat_svr.reshape(-1, 1))
    rmse_svr = np.sqrt(mean_squared_error(test_y, inv_yhat_svr))
    nrmse_svr = rmse_svr / test_y_std  # Normalize using std of test_y
    results['SVR'].append(nrmse_svr)
    
    # MLP
    mlp_model = MLPRegressor(random_state=i,**mlp_params)  # Add your pre-tuned parameters
    mlp_model.fit(s_train_X, s_train_y)
    yhat_mlp = mlp_model.predict(s_test_X)
    inv_yhat_mlp = y_scaler.inverse_transform(yhat_mlp.reshape(-1, 1))
    rmse_mlp = np.sqrt(mean_squared_error(test_y, inv_yhat_mlp))
    nrmse_mlp = rmse_mlp / test_y_std
    results['MLP'].append(nrmse_mlp)

    # Voting
    voting_model = VotingRegressor(estimators=[
        ('svr', SVR(**svr_params)),  # Add parameters
        ('mlp', MLPRegressor(**mlp_params))  # Add parameters
    ])
    voting_model.fit(s_train_X, s_train_y)
    yhat_voting = voting_model.predict(s_test_X)
    inv_yhat_voting = y_scaler.inverse_transform(yhat_voting.reshape(-1, 1))
    rmse_voting = np.sqrt(mean_squared_error(test_y, inv_yhat_voting))
    nrmse_voting = rmse_voting / test_y_std
    results['SVR_MLP'].append(nrmse_voting)
    
    # Enhanced Model (DenseAttNet)
    enhanced_model = create_enhanced_model(s_train_X.shape[1])
    train_loss_stop = EarlyStopping(
    monitor='loss',
    patience=25,  
    min_delta=0.001,
    restore_best_weights=True
)
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=10, 
    min_lr=1e-6,
    cooldown=3,  
    verbose=1
)
    
    enhanced_model.fit(
    s_train_X, s_train_y,
    epochs=1000,
    batch_size=128,
    callbacks=[train_loss_stop,lr_scheduler],  
    verbose=1
)
    yhat_enhanced = enhanced_model.predict(s_test_X)
    inv_yhat_enhanced = y_scaler.inverse_transform(yhat_enhanced)
    rmse_enhanced = np.sqrt(mean_squared_error(test_y, inv_yhat_enhanced))
    nrmse_enhanced = rmse_enhanced / test_y_std
    results['DenseAttNet'].append(nrmse_enhanced)

In [19]:
# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df

,SVR,MLP,SVR_MLP,DenseAttNet,RF,DT,GBR,XGB
0,0.448172,0.443032,0.439925,0.438748,0.491100,0.574005,0.460129,0.470092
1,0.526151,0.536203,0.526745,0.525959,0.550679,0.627356,0.521046,0.529688
2,0.470094,0.468797,0.454141,0.459671,0.486538,0.579278,0.464549,0.469467
3,0.500811,0.509878,0.497869,0.499106,0.516228,0.624948,0.499567,0.517126
4,0.514535,0.491923,0.509506,0.492261,0.531717,0.579961,0.520020,0.537900
5,0.488505,0.501195,0.489160,0.485373,0.519018,0.563847,0.489828,0.512735
6,0.461167,0.487411,0.463060,0.478393,0.468041,0.552114,0.475419,0.482257
7,0.487803,0.488702,0.487927,0.488428,0.515696,0.628377,0.488111,0.515190
8,0.519104,0.521594,0.507338,0.510047,0.565208,0.621041,0.548121,0.540804
9,0.492837,0.508833,0.488381,0.482617,0.507964,0.613874,0.480581,0.505352


In [26]:
# 6. Statistical comparison of NRMSE
print("\n=== Final NRMSE Results ===")
for model, nrmses in results.items():
    print(f"{model} NRMSE: Mean={np.mean(nrmses):.4f}, SD={np.std(nrmses):.4f}")



=== Final NRMSE Results ===
SVR NRMSE: Mean=0.4938, SD=0.0264
MLP NRMSE: Mean=0.4994, SD=0.0281
SVR_MLP NRMSE: Mean=0.4902, SD=0.0280
DenseAttNet NRMSE: Mean=0.4879, SD=0.0252
RF NRMSE: Mean=0.5164, SD=0.0290
DT NRMSE: Mean=0.6026, SD=0.0379
GBR NRMSE: Mean=0.4959, SD=0.0272
XGB NRMSE: Mean=0.5104, SD=0.0288


In [ ]:
print("\n=== Statistical Comparisons (NRMSE) ===")
models = list(results.keys())
for i in range(len(models)):
    for j in range(i+1, len(models)):
        # Paired t-test
        t_stat, p_value_t = stats.ttest_rel(results[models[i]], results[models[j]])
        # Wilcoxon signed-rank test
        w_stat, p_value_w = stats.wilcoxon(results[models[i]], results[models[j]])
        
        # Calculate mean difference (model i - model j)
        mean_diff = np.mean(results[models[i]]) - np.mean(results[models[j]])
        
        print(f"{models[i]} vs {models[j]}:")
        print(f"  Mean Δ: {mean_diff:.4f}")
        print(f"  t-test: t={t_stat:.4f}, p={p_value_t:.4f}")
        print(f"  Wilcoxon: W={w_stat}, p={p_value_w:.4f}")


=== Statistical Comparisons (NRMSE) ===
SVR vs MLP:
  Mean Δ: -0.0056
  t-test: t=-2.2087, p=0.0397
  Wilcoxon: W=51.0, p=0.0441
SVR vs SVR_MLP:
  Mean Δ: 0.0036
  t-test: t=3.2479, p=0.0042
  Wilcoxon: W=31.0, p=0.0042
SVR vs DenseAttNet:
  Mean Δ: 0.0059
  t-test: t=2.8756, p=0.0097
  Wilcoxon: W=37.0, p=0.0094
SVR vs RF:
  Mean Δ: -0.0226
  t-test: t=-8.7328, p=0.0000
  Wilcoxon: W=0.0, p=0.0000
SVR vs DT:
  Mean Δ: -0.1088
  t-test: t=-18.2280, p=0.0000
  Wilcoxon: W=0.0, p=0.0000
SVR vs GBR:
  Mean Δ: -0.0020
  t-test: t=-0.8540, p=0.4037
  Wilcoxon: W=91.0, p=0.6215
SVR vs XGB:
  Mean Δ: -0.0166
  t-test: t=-7.3630, p=0.0000
  Wilcoxon: W=3.0, p=0.0000
MLP vs SVR_MLP:
  Mean Δ: 0.0092
  t-test: t=4.0874, p=0.0006
  Wilcoxon: W=20.0, p=0.0007
MLP vs DenseAttNet:
  Mean Δ: 0.0115
  t-test: t=7.1655, p=0.0000
  Wilcoxon: W=2.0, p=0.0000
MLP vs RF:
  Mean Δ: -0.0170
  t-test: t=-4.5777, p=0.0002
  Wilcoxon: W=16.0, p=0.0003
MLP vs DT:
  Mean Δ: -0.1032
  t-test: t=-17.8860, p=0.0000

In [25]:
# Additional: Calculate confidence intervals for mean NRMSE differences
print("\n=== Confidence Intervals for Differences ===")
for i in range(len(models)):
    for j in range(i+1, len(models)):
        differences = np.array(results[models[i]]) - np.array(results[models[j]])
        mean_diff = np.mean(differences)
        ci = stats.t.interval(0.95, len(differences)-1, 
                             loc=mean_diff, 
                             scale=stats.sem(differences))
        print(f"{models[i]} - {models[j]}: Mean Δ = {mean_diff:.4f}, 95% CI = [{ci[0]:.4f}, {ci[1]:.4f}]")


=== Confidence Intervals for Differences ===
SVR - MLP: Mean Δ = -0.0056, 95% CI = [-0.0108, -0.0003]
SVR - SVR_MLP: Mean Δ = 0.0036, 95% CI = [0.0013, 0.0059]
SVR - DenseAttNet: Mean Δ = 0.0059, 95% CI = [0.0016, 0.0102]
SVR - RF: Mean Δ = -0.0226, 95% CI = [-0.0280, -0.0172]
SVR - DT: Mean Δ = -0.1088, 95% CI = [-0.1213, -0.0963]
SVR - GBR: Mean Δ = -0.0020, 95% CI = [-0.0069, 0.0029]
SVR - XGB: Mean Δ = -0.0166, 95% CI = [-0.0213, -0.0119]
MLP - SVR_MLP: Mean Δ = 0.0092, 95% CI = [0.0045, 0.0138]
MLP - DenseAttNet: Mean Δ = 0.0115, 95% CI = [0.0081, 0.0148]
MLP - RF: Mean Δ = -0.0170, 95% CI = [-0.0248, -0.0092]
MLP - DT: Mean Δ = -0.1032, 95% CI = [-0.1153, -0.0911]
MLP - GBR: Mean Δ = 0.0035, 95% CI = [-0.0035, 0.0106]
MLP - XGB: Mean Δ = -0.0110, 95% CI = [-0.0177, -0.0044]
SVR_MLP - DenseAttNet: Mean Δ = 0.0023, 95% CI = [-0.0015, 0.0061]
SVR_MLP - RF: Mean Δ = -0.0262, 95% CI = [-0.0326, -0.0198]
SVR_MLP - DT: Mean Δ = -0.1124, 95% CI = [-0.1253, -0.0995]
SVR_MLP - GBR: Mean Δ